# Wheel Loader Service Request
This is a KWH HMI which allowes to request the wheel loader's location. Everyone can log in with their access data and assign this notebook to their HMI. To do this, simply enter your own HMI with the corresponding secret as the client in this script. Running this notebook, you will authorize your HMI to call up the wheel loader's location. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [5]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests

This Notebook acts as a client in terms of the S³I. In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [6]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Assign a client to this notebook. (The id of your HMI)")
clientId = input('[S3I]: Please enter the client id:')
clientSecret = getpass.getpass('[S3I]: Please enter the secret:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Client id and secret are set")

[S3I][2020-05-24 22:14:52]: Assign a client to this notebook. (The id of your HMI)
[S3I]: Please enter the client id:s3i:84c6a8b5-88eb-4759-a9b0-3abc5f9079c7
[S3I]: Please enter the secret:········
[S3I][2020-05-24 22:15:11]: Client id and secret are set


Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [7]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id=clientId, client_secret=clientSecret, username=username, password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

[S3I][2020-05-24 22:15:11]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)
[S3I]: Please enter your username:user1
[S3I]: Please enter the password:········
[S3I][2020-05-24 22:15:19]: Your credentials are sent to S3I IdentityProvider.
[S3I][2020-05-24 22:15:19]: Token received user1 logged in.


After the token has been received, it can be used to query the endpoint of the wheel loader. To query the wheel loader in the S³I Directory, you must have the rights to view the wheel loader. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the wheel loader, the enpoint of this HMI is also needed to add as "ReplyToEnpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may has changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [8]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoint = s3i_directory.queryThingIDBased(clientId+"/attributes/allEndpoints")[0]
    
''' query the wheel loaders's Id and endpoint
'''
wheelloaderID = s3i_directory.queryAttributeBased(
    "name", "wheelloader")[0]["thingId"]

wheelloaderServiceEndpoint = s3i_directory.queryEndpointService(
    wheelloaderID, "currentJobsDescription")

receivers = [wheelloaderID]
receiver_endpoints = wheelloaderServiceEndpoint

[S3I][2020-05-24 22:15:19]: Authentication with Token in S3I Directory successful


After querying the enpoints, the message is prepared. Since the location is one value of the wheel loader, a *GetValueRequest* is instantiated and filled in.

In [9]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Prepare the GetValueRequest.")
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the wheel loader (e.g., attributes/features/ml40::Location): \n')

msg_uuid = "s3i:" + str(uuid.uuid4())

servReq = s3i.GetValueRequest()
servReq.fillGetValueRequest(senderUUID=clientId, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                                attributePath=attributePath, msgUUID=msg_uuid)
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: GetValueRequest prepared:", servReq.msg)

[S3I][2020-05-24 22:15:19]: Prepare the GetValueRequest.
[S3I]: please enter the attribute path of the value you want to receive from the wheel loader (e.g., attributes/features/ml40::Location): 
attributes/features/ml40::Location
[S3I][2020-05-24 22:15:45]: GetValueRequest prepared: {'sender': 's3i:84c6a8b5-88eb-4759-a9b0-3abc5f9079c7', 'identifier': 's3i:2b2bed69-ee4f-4b05-8da6-32c0fb4aadb5', 'receivers': ['s3i:7f654f13-11ac-413d-841c-10a8e431fc35'], 'messageType': 'getValueRequest', 'replyToEndpoint': 's3ibs://s3i:84c6a8b5-88eb-4759-a9b0-3abc5f9079c7', 'attributePath': 'attributes/features/ml40::Location'}


The HMI uses the token to establish a connection to the broker and sends the request to the wheel loader's endpoint. It then checks incoming responses.

In [10]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Sending the GetValueRequest to the wheel loader")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
#print(receiver_endpoints)
response = requests.post(url="https://broker.s3i.vswf.dev/"+receiver_endpoints[0] ,
                                data=json.dumps(servReq.msg), headers=headers)
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]:", response.text)

[S3I][2020-05-24 22:15:45]: Sending the GetValueRequest to the wheel loader
['s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35']
[S3I][2020-05-24 22:15:45]: [{"endpoint": "s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35", "status": "Sending message succeeded"}]


In [11]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Check the wheel loaders response")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
json_acceptable_string = response.text.replace("'", "\"")

if json_acceptable_string:
    response_json = json.loads(json_acceptable_string)
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: The wheel loader responded with:")
    print(30*" "+"Longitude: ", response_json["value"]["longitude"])
    print(30*" "+"Latitude: ", response_json["value"]["latitude"])
else:
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: The wheel did not respond yet.")

[S3I][2020-05-24 22:15:45]: Check the wheel loaders response
[S3I][2020-05-24 22:15:46]: The wheel loader responded with:
                              Longitude:  7.97917
                              Latitude:  51.45278
